In [1]:
import CS_Indicators as CS
from brix import Indicator, Handler
import geopandas as gpd
import pandas as pd

In [2]:
table_name='epa_test'

### Load the Saved Data

In [3]:
geogrid=gpd.read_file('tables/{}/geogrid.geojson'.format(table_name))
zones=gpd.read_file('tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype(int)
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('tables/{}/simpop_df.csv'.format(table_name))

In [6]:
from brix import Indicator, Handler
import OpenCity
import PreCompOsmNet
import statsmodels
from statsmodels.distributions.empirical_distribution import ECDF
import geopandas as gpd
import urllib
import json
import requests
import math
import osmnx
from shapely.geometry import Point, shape
import datetime
from scipy import spatial
import numpy as np
import sys
import pandas as pd

In [10]:
test={'id_1':
     {'score_1': 1, 'score_2': 2},
     'id_2':
     {'score_1': 2, 'score_2': 3}}
test

{'id_1': {'score_1': 1, 'score_2': 2}, 'id_2': {'score_1': 2, 'score_2': 3}}

In [155]:
pd.DataFrame.from_dict(test)

,score_1,score_2
id_1,1,2
id_2,2,3


In [177]:
def aggregate_types_over_grid(geogrid_data, side_length, type_def):
    cell_area=side_length*side_length
    aggregated={}  
    for cell in geogrid_data:
        name=cell['name']
        type_info=type_def[name]
        height=cell['height']
        if isinstance(height, list):
            height=height[-1]
        if 'sqm_pperson' in type_info:
            sqm_pperson=type_info['sqm_pperson']
        else:
            sqm_pperson=50
        total_capacity=height*cell_area/sqm_pperson
        if name in aggregated:
            aggregated[name]+=total_capacity
        else:
            aggregated[name]=total_capacity
    return aggregated

def pop_one_cell(cell, side_length, type_def):
    name=cell['name']
    type_info=type_def[name]
    height=cell['height']
    cell_area=side_length*side_length
    if isinstance(height, list):
        height=height[-1]
    if 'sqm_pperson' in type_info:
        sqm_pperson=type_info['sqm_pperson']
    else:
        sqm_pperson=50
    total_capacity=height*cell_area/sqm_pperson
    if type_info['NAICS'] is None:
        emp=0
    else:
        emp=total_capacity*sum(type_info['NAICS'].values())
    if type_info['LBCS'] is None:
        res=0
    else:
        res=total_capacity*sum([type_info['LBCS'][code] for code in type_info['LBCS'] if code.startswith('1')])
    return res, emp
    
        

def aggregate_attributes_over_grid(agg_types, attribute, side_length, type_def, digits=None):
    # TODO: eliminate repetition with previous function
    aggregated={}
    for cell_type in agg_types:
        type_info=type_def[cell_type]
        if type_info[attribute] is not None:
            total_capacity=agg_types[cell_type]
            for code in type_info[attribute]:
                if digits==None:
                    code_digits=code
                else:
                    code_digits=code[0:digits]
                attr_capacity=total_capacity*type_info[attribute][code]
                if code_digits in aggregated:
                    aggregated[code_digits]+= attr_capacity
                else:
                    aggregated[code_digits]= attr_capacity
    return aggregated

def get_central_nodes(geodf, G):
    """ takes a geodf and returns a list of nodes closest to their centroids
    returns both the nearest nodes and the distance
    """
    geodf_projected=osmnx.projection.project_gdf(geodf)
    projected_centroids=geodf_projected['geometry'].centroid.values
    projected_centroids_lst=[[c.x, c.y] for c in projected_centroids]
    G_proj=osmnx.projection.project_graph(G, geodf_projected.crs)
    G_proj_coordinates={node: [G_proj.nodes[node]['x'], G_proj.nodes[node]['y']] for node in G_proj.nodes}
    node_order=[node for node in G_proj_coordinates]
    nodes_kdtree=spatial.KDTree([G_proj_coordinates[node] for node in node_order])
    dist, ind_nearest=nodes_kdtree.query(projected_centroids_lst)
    nearest_nodes=[node_order[ind] for ind in ind_nearest]
    return nearest_nodes, dist


import copy
class Proximity_Indicator(Indicator):
    def setup(self, zones, geogrid, buffer=1200):
        print('Setting up Proximity Indicator')
        self.zone_to_node_tolerance=500
        self.grid_to_node_tolerance=100
        self.buffer=buffer
        self.zones=zones
        self.indicator_type = 'hybrid'
        self.geogrid=geogrid
        self.overlapping_geoids=list(zones.loc[zones['sim_area']].index)
        self.all_site_ids=self.overlapping_geoids+list(range(len(geogrid_data)))  
        self.get_graph_reference_area()
        self.get_reachable_geoms_from_all()
        self.naics_codes=[col for col in zones.columns if 'naics' in col]
    
        print('\t Getting central nodes')
        zones_nearest_nodes, zones_nearest_dist= get_central_nodes(self.zones, self.ref_G)
        self.zones['central_node']=zones_nearest_nodes
        self.zones['nearest_dist']=zones_nearest_dist
        
        grid_nearest_nodes, grid_nearest_dist= get_central_nodes(self.geogrid, self.ref_G)
        self.geogrid['central_node']=grid_nearest_nodes
        self.geogrid['nearest_dist']=grid_nearest_dist
        self.calculate_baseline_scores()
            
            
    def make_ego_graph_around_geometry(self, zone, tolerance):
        """
        For geometries within the buffered geogrid only.
        Returns the graph within a walkable distance of the centre of the zone
        """
        if zone['nearest_dist']<tolerance:
            sub_graph = osmnx.graph.nx.ego_graph(self.ref_G, zone['central_node'], radius=1200, distance='length')
        else:
            sub_graph = osmnx.graph.nx.Graph()
        return sub_graph
    
    def get_graph_reference_area(self):
        reference_zones=self.zones.loc[self.zones['reference_area']]
        print('\t Downloading graph for reference area')
        reference_zone_graph=self.get_network_around_geom_buffered(reference_zones)
        self.ref_G=reference_zone_graph
        
    def calculate_baseline_scores(self):
        # TODO: should use the get_reachable_geoms function?
        print('\t Calculating baseline scores')
#         self.base_scores={'walkable_{}'.format(x): [] for x in [
#             'employment', 'housing', 'healthcare', 'hospitality', 'shopping']}
        base_scores={}
        self.base_attributes={}
        self.score_ecdfs={}
        stats_to_aggregate=[col for col in self.zones.columns if (('res_' in col) or ('emp_' in col))]
        # get the baseline reachable attributes and scores for every zone
        for ind, row in self.zones.loc[self.zones['reference_area']].iterrows():
            reachable_zones=self.zone_to_reachable[ind]['zones']
            self.base_attributes[ind]=self.zones.loc[reachable_zones][stats_to_aggregate].sum().to_dict()
            self.base_attributes[ind]['source_res']=row['res_total']
            self.base_attributes[ind]['source_emp']=row['emp_total']
            # get scores for individual zones- weighting cancels out
            base_scores[ind]=self.attributes_to_scores([self.base_attributes[ind]])
            
        # Create the ECDFs for each score using only the zones (not the grid cells
        self.base_zones_scores=pd.DataFrame.from_dict(base_scores, orient='index')
        for score in self.base_zones_scores.columns:
            print(score)
            base_scores_no_nan=[x for x in self.base_zones_scores[score] if x==x]
            self.score_ecdfs[score]=ECDF(base_scores_no_nan)
            
        # get weighted scores across the simulation area zones 
        # (ignore the grid which is empty in reference and therefore would be weighted zero)
        ref_scores=self.attributes_to_scores([self.base_attributes[ind] for ind in self.overlapping_geoids])
        self.ref_ind=self.normalise_ind(ref_scores)
            
        # get the base reachable attributes for every grid cell location
        for i_c in range(len(self.geogrid)):
            reachable_zones=self.grid_to_reachable[i_c]['zones']
            self.base_attributes[i_c]=self.zones.loc[reachable_zones][stats_to_aggregate].sum().to_dict()
            
            
    def get_reachable_geoms(self, zone, tolerance):
        """
        find all grid cells and all zones reachable from a geometry
        """
        sub_graph=self.make_ego_graph_around_geometry(zone, tolerance)
        sub_graph_nodes=sub_graph.nodes(data=False)
        reachable_zones= list(self.zones.loc[
                ((self.zones['central_node'].isin(list(sub_graph_nodes)))&
                 (self.zones['nearest_dist']<self.zone_to_node_tolerance))
                ].index.values)
        reachable_grid_cells=list(self.geogrid.loc[
                ((self.geogrid['interactive']=='Web')&
                    (self.geogrid['central_node'].isin(list(sub_graph_nodes)))&
                    (self.geogrid['nearest_dist']<self.grid_to_node_tolerance))
                ].index.values)
        return {'zones': reachable_zones, 'cells': reachable_grid_cells}
    
    def get_reachable_geoms_from_all(self):
        """
        For every grid cell and every zone which intersects the grid:
        find the reachable zones and reachable grid cells
        """
        print('\t Finding all reachable geometries from each geometry')
        self.grid_to_reachable, self.zone_to_reachable={}, {}
        for ind, row in self.geogrid.iterrows():
            self.grid_to_reachable[ind]=self.get_reachable_geoms(row, self.grid_to_node_tolerance)
        for ind, row in self.zones.loc[self.zones['reference_area']].iterrows():
            self.zone_to_reachable[ind]=self.get_reachable_geoms(row, self.zone_to_node_tolerance)
        # create a reverse lookup to map from eacg grid cell to the cells from which it is reachable
        self.grid_to_reverse_reachable={}
        for i, row_i in self.geogrid.iterrows():
            self.grid_to_reverse_reachable[i]={'zones': [], 'cells': []}
            for j, row_j in self.geogrid.iterrows():
                if i in self.grid_to_reachable[j]['cells']:
                    self.grid_to_reverse_reachable[i]['cells'].append(j)
            for ind_z, row_z in self.zones.loc[self.zones['reference_area']].iterrows():
                if i in self.zone_to_reachable[ind_z]['cells']:
                    self.grid_to_reverse_reachable[i]['zones'].append(ind_z)                    
            
    def attributes_to_scores(self, attributes):        
        total_emp=sum([s['source_emp'] for s in attributes])
        total_res=sum([s['source_res'] for s in attributes])

        scores={}

        scores['walkable_housing']=sum([s['source_emp']*s['res_total'] for s in attributes])/total_emp
        scores['walkable_employment']=sum([s['source_res']*s['emp_total'] for s in attributes])/total_res
        scores['walkable_healthcare']=sum([s['source_res']*s['emp_naics_62'] for s in attributes])/total_res
        scores['walkable_hospitality']=sum([s['source_res']*s['emp_naics_72'] for s in attributes])/total_res
        scores['walkable_shopping']=sum([s['source_res']*s['emp_naics_44-45'] for s in attributes])/total_res
        return scores
    
    def return_indicator(self, geogrid_data):
        start_ind_calc=datetime.datetime.now()
        # make copy of base_scores
        new_attributes=self.get_new_reachable_attributes(geogrid_data)
        new_attributes_site= [new_attributes[ind] for ind in self.all_site_ids]
        new_scores=self.attributes_to_scores(new_attributes_site)
        new_ind=self.normalise_ind(new_scores)
        outputs=[]
        for ind_name in new_scores:
            outputs.append({'name': ind_name ,
                            'value': new_ind[ind_name],
                            'raw_value': new_scores[ind_name],
                            'ref_value': self.ref_ind[ind_name],
                            'viz_type': self.viz_type})
        end_ind_calc=datetime.datetime.now()
        
        new_attributes_site= [new_attributes[i_c] for i_c in range(len(geogrid_data))]
        heatmap=self.compute_heatmaps(new_attributes_site)
        
        end_hm_calc=datetime.datetime.now()
        
        print('Prox Ind: {}'.format(end_ind_calc-start_ind_calc))
        print('Prox HM: {}'.format(end_hm_calc-end_ind_calc))
        
        return {'heatmap':heatmap,'numeric':outputs}
    
    
    def get_new_reachable_attributes(self, geogrid_data):
        new_attributes=copy.deepcopy(self.base_attributes)
        side_length=geogrid_data.get_geogrid_props()['header']['cellSize']
        cell_area=side_length*side_length
        type_def=geogrid_data.get_type_info()
        for i_c, cell in enumerate(geogrid_data):
            name=cell['name']
            height=cell['height']
            if isinstance(height, list):
                height=height[-1]
            type_info = type_def[name] 
            if 'sqm_pperson' in type_info:
                sqm_pperson=type_info['sqm_pperson']
            else:
                sqm_pperson=50
            total_capacity=height*cell_area/sqm_pperson
            agg_naics=aggregate_attributes_over_grid(
                {name: total_capacity}, 'NAICS', side_length, type_def, digits=2)
            agg_lbcs=aggregate_attributes_over_grid(
                {name: total_capacity}, 'LBCS', side_length, type_def, digits=1)

            added_attributes={}
            cell_employment=sum(agg_naics.values())
            new_attributes[i_c]['source_emp']=cell_employment
            added_attributes['emp_total']=cell_employment

            if '1' in agg_lbcs:
                cell_population=agg_lbcs['1']
            else:
                cell_population=0

            added_attributes['res_total']=cell_population
            new_attributes[i_c]['source_res']=cell_population

            for combined_code in self.naics_codes:
                naics_codes=combined_code.split('naics_')[1].split('-')
                for code in naics_codes:
                    if code in agg_naics:
                        if code in added_attributes:
                            added_attributes[combined_code]+=agg_naics[code]
                        else:
                            added_attributes[combined_code]=agg_naics[code]
            # the newly attributes are added to every zone and cell which can reach this cell   
            reverse_reachable=self.grid_to_reverse_reachable[i_c]
            for ind_z in reverse_reachable['zones']:
                for attr in added_attributes:
                    new_attributes[ind_z][attr]+=added_attributes[attr]
            for j_c in reverse_reachable['cells']:
                for attr in added_attributes:
                    new_attributes[j_c][attr]+=added_attributes[attr] 
        return new_attributes

    def normalise_ind(self, raw_ind):
        norm_ind={}
        for ind_name in raw_ind:
            norm_ind[ind_name]=self.score_ecdfs[ind_name](raw_ind[ind_name])       
        return norm_ind


    def compute_heatmaps(self, grid_reachable_area_stats):
        max_scores={score: self.base_zones_scores[score].max() for score in self.base_zones_scores}
        features=[]
        heatmap={'type': 'FeatureCollection',
                 'properties': ['housing', 'employment', 'healthcare', 'hospitality', 'shopping']}
        x_centroid_list, y_centroid_list=self.geogrid['x_centroid'], self.geogrid['y_centroid']
        for i_c, cell_stats in enumerate(grid_reachable_area_stats):
            features.append({
              "type": "Feature",
              "properties": [min((cell_stats['res_total']/max_scores['walkable_housing']), 1), 
                             min((cell_stats['emp_total']/max_scores['walkable_employment']), 1),
                             min((cell_stats['emp_naics_62']/max_scores['walkable_healthcare']), 1), 
                             min((cell_stats['emp_naics_72']/max_scores['walkable_hospitality']), 1), 
                             min((cell_stats['emp_naics_44-45']/max_scores['walkable_shopping']), 1)],
              "geometry": {
                "type": "Point",
                "coordinates": [
                  x_centroid_list[i_c],
                  y_centroid_list[i_c]
                ]
              }
            })
        heatmap['features']=features
        return heatmap
                        
#     def aggregate_reachable_attributes_one_source(self, zones, cells, geogrid_data=None):
#         stats_to_aggregate=[
#                 col for col in self.zones.columns if (('res_' in col) or ('emp_' in col))]
#         reachable_area_stats=dict(self.zones.loc[zones, stats_to_aggregate].sum())
#         if geogrid_data is not None:
#             geogrid_data_reachable=[geogrid_data[c] for c in cells]
#             side_length=geogrid_data.get_geogrid_props()['header']['cellSize']
#             type_def=geogrid_data.get_type_info()
#             agg_types=aggregate_types_over_grid(geogrid_data_reachable, side_length=side_length, type_def=type_def)
#             agg_naics=aggregate_attributes_over_grid(agg_types, 'NAICS', side_length=side_length, type_def=type_def, digits=2)
#             agg_lbcs=aggregate_attributes_over_grid(agg_types, 'LBCS', side_length=side_length, type_def=type_def, digits=1)
            
#             # update total residential and total employment
#             add_emp=sum(agg_naics.values())
#             if '1' in agg_lbcs:
#                 add_res=agg_lbcs['1']
#             else:
#                 add_res=0    
#             reachable_area_stats['res_total']+=add_res
#             reachable_area_stats['emp_total']+=add_emp
            
#             # update employment for each NAICS code
#             for col in reachable_area_stats:
#                 if 'naics' in col:
#                     col_naics_codes=col.split('naics_')[1].split('-')
#                     for code in col_naics_codes:
#                         if code in agg_naics:
#                             reachable_area_stats[col]+=agg_naics[code]                             
#             # update residential types
#             if 'Residential Low Income' in agg_types:
#                 reachable_area_stats['res_income_low']+=agg_types['Residential Low Income']
#             if 'Residential Med Income' in agg_types:
#                 reachable_area_stats['res_income_mid']+=agg_types['Residential Med Income']
#             if 'Residential High Income' in agg_types:
#                 reachable_area_stats['res_income_high']+=agg_types['Residential High Income'] 
#         return reachable_area_stats
                
#     def aggregate_reachable_attributes_all_sources(self, geogrid_data=None):
#         zone_reachable_area_stats, grid_reachable_area_stats=[], []
#         for ind, row in self.zones.loc[self.overlapping_geoids].iterrows():
#             reachable_area_stats=self.aggregate_reachable_attributes_one_source(
#                 self.zone_to_reachable[ind]['zones'], self.zone_to_reachable[ind]['cells'], geogrid_data)
#             reachable_area_stats['source_pop']=row['res_total']
#             reachable_area_stats['source_emp']=row['emp_total']
#             reachable_area_stats['GEOID']=ind
#             zone_reachable_area_stats.append(reachable_area_stats)
#         if geogrid_data is not None:
#             side_length=geogrid_data.get_geogrid_props()['header']['cellSize']
#             type_def=geogrid_data.get_type_info()
#             for i_c, cell in enumerate(geogrid_data):
#                 reachable_area_stats=self.aggregate_reachable_attributes_one_source(
#                     self.grid_to_reachable[i_c]['zones'], self.grid_to_reachable[i_c]['cells'], geogrid_data)
#                 res, emp=pop_one_cell(cell, side_length, type_def)
#                 reachable_area_stats['source_pop']=res
#                 reachable_area_stats['source_emp']=emp
#                 reachable_area_stats['id']=i_c
#                 grid_reachable_area_stats.append(reachable_area_stats)
#         return zone_reachable_area_stats, grid_reachable_area_stats
    
#     def compute_heatmaps(self, grid_reachable_area_stats):
#         max_scores={score: max(self.base_scores[score]) for score in self.base_scores}
#         features=[]
#         heatmap={'type': 'FeatureCollection',
#                  'properties': ['housing', 'employment', 'healthcare', 'hospitality', 'shopping']}
#         x_centroid_list, y_centroid_list=self.geogrid['x_centroid'], self.geogrid['y_centroid']
#         for i_c, cell_stats in enumerate(grid_reachable_area_stats):
#             features.append({
#               "type": "Feature",
#               "properties": [min((cell_stats['res_total']/max_scores['walkable_housing']), 1), 
#                              min((cell_stats['emp_total']/max_scores['walkable_employment']), 1),
#                              min((cell_stats['emp_naics_62']/max_scores['walkable_healthcare']), 1), 
#                              min((cell_stats['emp_naics_72']/max_scores['walkable_hospitality']), 1), 
#                              min((cell_stats['emp_naics_44-45']/max_scores['walkable_shopping']), 1)],
#               "geometry": {
#                 "type": "Point",
#                 "coordinates": [
#                   x_centroid_list[i_c],
#                   y_centroid_list[i_c]
#                 ]
#               }
#             })
#         heatmap['features']=features
#         return heatmap
                 
#     def calculate_indicators(self, site_stats):
#         raw_ind={}
#         sum_all_source_pop=sum([s['source_pop'] for s in site_stats])
#         sum_all_source_emp=sum([s['source_emp'] for s in site_stats])
#         raw_ind['walkable_housing']=sum([s['source_emp']*s['res_total'] for s in site_stats])/sum_all_source_emp
#         raw_ind['walkable_employment']=sum([s['source_pop']*s['emp_total'] for s in site_stats])/sum_all_source_pop
#         raw_ind['walkable_healthcare']=sum([s['source_pop']*s['emp_naics_62'] for s in site_stats])/sum_all_source_pop
#         raw_ind['walkable_hospitality']=sum([s['source_pop']*s['emp_naics_72'] for s in site_stats])/sum_all_source_pop
#         raw_ind['walkable_shopping']=sum([s['source_pop']*s['emp_naics_44-45'] for s in site_stats])/sum_all_source_pop
        
#         norm_ind={}
#         for ind_name in raw_ind:
#             norm_ind[ind_name]=self.score_ecdfs[ind_name](raw_ind[ind_name])       
#         return {'raw': raw_ind, 'norm': norm_ind}
                  
#     def return_indicator(self, geogrid_data):
#         start_ind_calc=datetime.datetime.now()
#         zone_site_stats, grid_site_stats=self.aggregate_reachable_attributes_all_sources(geogrid_data)
#         new_ind=self.calculate_indicators(zone_site_stats + grid_site_stats)
        
#         base_zone_site_stats, base_grid_site_stats=self.aggregate_reachable_attributes_all_sources()
#         base_ind=self.calculate_indicators(base_zone_site_stats)
        
#         outputs=[]
#         for ind_name in new_ind['raw']:
#             outputs.append({'name': ind_name.replace('_', ' ').title(),
#                            'raw_value': new_ind['raw'][ind_name],
#                            'value': new_ind['norm'][ind_name],
#                            'ref_value': base_ind['norm'][ind_name]})
#         end_ind_calc=datetime.datetime.now()
        
#         heatmap=self.compute_heatmaps(grid_site_stats)
#         end_hm_calc=datetime.datetime.now()
        
#         print('Prox Ind: {}'.format(end_ind_calc-start_ind_calc))
#         print('Prox HM: {}'.format(end_hm_calc-end_ind_calc))
        
#         return {'heatmap':heatmap,'numeric':outputs}
      
    def get_network_around_geom_buffered(self, geom):
        """
        Creates a buffer around the geometry and downloads the graph for this area
        """
        geom_projected=osmnx.projection.project_gdf(geom)
        geom_projected_buffered=geom_projected.unary_union.buffer(self.buffer)

        geom_projected_buffered_gdf=gpd.GeoDataFrame(geometry=[geom_projected_buffered], crs=geom_projected.crs)
        geom_wgs_buffered_gdf=geom_projected_buffered_gdf.to_crs(geom.crs) 
        
        return osmnx.graph.graph_from_polygon(geom_wgs_buffered_gdf.iloc[0]['geometry'], network_type='walk')

In [178]:
p=Proximity_Indicator(zones, geogrid)

Setting up Proximity Indicator
	 Finding all reachable geometries from each geometry
	 Getting central nodes
	 Calculating baseline scores
walkable_housing
walkable_employment
walkable_healthcare
walkable_hospitality
walkable_shopping


In [179]:
self=p
H=Handler(table_name=table_name)
geogrid_data=H.get_geogrid_data()

In [180]:
test=self.return_indicator(geogrid_data)

Prox Ind: 0:00:00.022576
Prox HM: 0:00:00.004287


### Create and run the Cityscope model

In [184]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

In [185]:
d=CS.Density_Indicator(zones=zones)
p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)
m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df)
H.add_indicators([d, p, m])

Getting central nodes
Calculating baseline scores
Building Mobility System
	 getting graph
Buffer : 200
getting external roads
	 Simplified from 582 to 404 edges and 275 to 176 nodes
	 Pre-computing paths
Init simulation


/Volumes/GoogleDrive/My Drive/OpenCity/CS_Indicators.py:548: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=grid_zones['geometry'].centroid


Getting internal nodes
Finding closest nodes to every zone centroid
Computing zone-zone dist mat
Dens Ind: 0:00:00.004380
Prox Ind: 0:00:00.326049
Prox HM: 0:00:00.003732
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update


In [ ]:
H.listen()

Dens Ind: 0:00:00.005588
Prox Ind: 0:00:00.500149
Prox HM: 0:00:00.001819
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.012442
Prox Ind: 0:00:01.519960
Prox HM: 0:00:00.006455
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.010272
Prox Ind: 0:00:00.502899
Prox HM: 0:00:00.003377
Starting MM Update
Schedules and locations
Trip table
Route table


...

Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.009148
Prox Ind: 0:00:00.807176
Prox HM: 0:00:00.001888
Starting MM Update
Schedules and locations
Trip table
Route table
DeckGL


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.008284
Prox Ind: 0:00:00.510943
Prox HM: 0:00:00.002218
Starting MM Update
Schedules and locations
Trip table
Route table
DeckGL


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Finished MM Update
